Loading all the library

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from torch.cuda.amp import autocast, GradScaler
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from torchvision.models import resnet18, resnet50

In [33]:
import warnings
warnings.filterwarnings('ignore')


Random seed for reproducibility

In [34]:
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [35]:
class ResNetFashionMNIST(nn.Module):  # Renamed from ResNetMNIST
    """Wrapper for ResNet to handle 1-channel FashionMNIST images"""
    def __init__(self, model_type='resnet18', num_classes=10):
        super(ResNetFashionMNIST, self).__init__()
        
        if model_type == 'resnet18':
            self.model = resnet18(pretrained=False)
            # Modify first conv layer for 1-channel input
            self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        elif model_type == 'resnet50':
            self.model = resnet50(pretrained=False)
            self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify final layer for 10 classes
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)


In [36]:
def load_fashionmnist_dataset():  # Renamed function
    """Load FashionMNIST dataset only"""
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # ResNet expects at least 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Only FashionMNIST
    dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=True, download=True, transform=transform
    )
    test_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=transform
    )
    
    # Split train into train and validation (70%-10%-20%)
    train_size = int(0.7 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_dataset, val_dataset, _ = random_split(
        dataset, [train_size, val_size, test_size]
    )
    
    return train_dataset, val_dataset, test_dataset

In [37]:
def train_epoch(model, train_loader, optimizer, criterion, device, scaler, use_amp=True):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        if use_amp:
            with autocast():
                output = model(data)
                loss = criterion(output, target)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(output, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return running_loss / len(train_loader), accuracy

In [38]:
def validate(model, val_loader, criterion, device, use_amp=True):
    """Validate the model"""
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            
            if use_amp:
                with autocast():
                    output = model(data)
                    loss = criterion(output, target)
            else:
                output = model(data)
                loss = criterion(output, target)
            
            val_loss += loss.item()
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return val_loss / len(val_loader), accuracy


In [39]:
def test(model, test_loader, device, use_amp=True):
    """Test the model"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            if use_amp:
                with autocast():
                    output = model(data)
            else:
                output = model(data)
            
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

In [54]:
def run_experiment(model_type, batch_size, optimizer_name, lr, 
                   pin_memory=False, epochs=10, use_amp=True):
    """Run a complete experiment for FashionMNIST"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load FashionMNIST dataset
    train_dataset, val_dataset, test_dataset = load_fashionmnist_dataset()
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                             pin_memory=pin_memory, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                           pin_memory=pin_memory, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            pin_memory=pin_memory, num_workers=2)
    
    # Create model
    model = ResNetFashionMNIST(model_type=model_type, num_classes=10).to(device)
    
    # Define loss function
    criterion = nn.CrossEntropyLoss()
    
    # Define optimizer
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    else:  # Adam
        optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Mixed precision training
    scaler = GradScaler() if use_amp else None
    
    # Training loop
    train_times = []
    epoch_details = []  # Store epoch details
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, optimizer, criterion, device, 
            scaler, use_amp
        )
        
        # Validate
        val_loss, val_acc = validate(model, val_loader, criterion, device, use_amp)
        
        epoch_time = time.time() - start_time
        train_times.append(epoch_time)
        
        # Store epoch details
        epoch_details.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc * 100,  # Convert to percentage
            'val_loss': val_loss,
            'val_acc': val_acc * 100,      # Convert to percentage
            'time': epoch_time
        })
        
        # Print epoch progress
        print(f"  Epoch {epoch+1}/{epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%, "
              f"Time: {epoch_time:.2f}s")
    
    # Test
    test_acc = test(model, test_loader, device, use_amp)
    avg_train_time = np.mean(train_times)
    
    # Print test results
    print(f"  Test Accuracy: {test_acc*100:.2f}%")
    print(f"  Average training time per epoch: {avg_train_time:.2f}s")
    
    return test_acc * 100, avg_train_time, epoch_details

In [57]:
def run_all_experiments():
    """Run all FashionMNIST experiments with epochs 3 and 5 only"""
    results = []
    all_epoch_details = []  # Store all epoch details
    
    # Hyperparameter combinations
    batch_sizes = [16, 32]  # List of batch sizes to iterate over
    optimizers = ['SGD', 'Adam']
    learning_rates = [0.001, 0.0001]  
    models = ['resnet18', 'resnet50']
    epochs_list = [2, 3]
    
    total_experiments = len(batch_sizes) * len(optimizers) * len(learning_rates) * len(models) * len(epochs_list)
    print(f"Running FashionMNIST experiments with epochs 3 and 5 only...")
    print(f"Total experiments to run: {total_experiments}")
    
    experiment_count = 0
    
    for batch_size in batch_sizes:  # Iterate over each batch size
        for optimizer_name in optimizers:
            for lr in learning_rates:
                for model_type in models:
                    for epochs in epochs_list:
                        experiment_count += 1
                        
                        print(f"\n{'='*70}")
                        print(f"EXPERIMENT {experiment_count}/{total_experiments}")
                        print(f"{'='*70}")
                        print(f"Configuration:")
                        print(f"  Model: {model_type}")
                        print(f"  Batch Size: {batch_size}")
                        print(f"  Optimizer: {optimizer_name}")
                        print(f"  Learning Rate: {lr}")
                        print(f"  Epochs: {epochs}")
                        print('='*70)
                        
                        try:
                            pin_memory = torch.cuda.is_available()
                            
                            # CORRECTED: Removed dataset_name parameter
                            test_acc, train_time, epoch_details = run_experiment(
                                model_type=model_type,
                                batch_size=batch_size,
                                optimizer_name=optimizer_name,
                                lr=lr,
                                pin_memory=pin_memory,
                                epochs=epochs,
                                use_amp=True
                            )
                            
                            # Add epoch details with configuration info
                            for epoch_detail in epoch_details:
                                epoch_detail.update({
                                    'exp_id': experiment_count,
                                    'model': model_type,
                                    'batch_size': batch_size,
                                    'optimizer': optimizer_name,
                                    'learning_rate': lr,
                                    'total_epochs': epochs
                                })
                                all_epoch_details.append(epoch_detail)
                            
                            # Store final results
                            results.append({
                                'Dataset': 'FashionMNIST',
                                'Model': model_type,
                                'Batch_Size': batch_size,
                                'Optimizer': optimizer_name,
                                'Learning_Rate': lr,
                                'Epochs': epochs,
                                'Test_Accuracy_%': test_acc,
                                'Avg_Train_Time_s': train_time
                            })
                            
                            print(f"\n✓ Experiment {experiment_count} completed successfully!")
                            print(f"  Final Test Accuracy: {test_acc:.2f}%")
                            print(f"  Avg Time per Epoch: {train_time:.2f}s")
                            
                        except Exception as e:
                            print(f"\n✗ Error in experiment {experiment_count}: {e}")
                            import traceback
                            traceback.print_exc()
                            continue
    
    # Create DataFrames
    df_results = pd.DataFrame(results)
    df_epoch_details = pd.DataFrame(all_epoch_details)
    
    # Save detailed epoch information
    df_epoch_details.to_csv('epoch_details.csv', index=False)
    df_results.to_csv('fashionmnist_results.csv', index=False)
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"ALL EXPERIMENTS COMPLETED")
    print(f"{'='*70}")
    print(f"Total successful experiments: {len(results)}/{total_experiments}")
    print(f"Results saved to: fashionmnist_results.csv")
    print(f"Epoch details saved to: epoch_details.csv")
    
    # Display epoch details summary
    if not df_epoch_details.empty:
        print(f"\n{'='*70}")
        print(f"EPOCH DETAILS SUMMARY")
        print(f"{'='*70}")
        
        # Show first few experiments' epoch details
        for exp_id in sorted(df_epoch_details['exp_id'].unique())[:3]:  # Show first 3 experiments
            exp_data = df_epoch_details[df_epoch_details['exp_id'] == exp_id]
            config = exp_data.iloc[0]
            
            print(f"\nExperiment {exp_id}: {config['model']}, BS={config['batch_size']}, "
                  f"{config['optimizer']}, LR={config['learning_rate']}")
            
            print(f"{'Epoch':<6} {'Train Loss':<12} {'Train Acc%':<12} "
                  f"{'Val Loss':<12} {'Val Acc%':<12} {'Time(s)':<10}")
            print(f"{'-'*70}")
            
            for _, row in exp_data.iterrows():
                print(f"{row['epoch']:<6} {row['train_loss']:<12.4f} {row['train_acc']:<12.2f} "
                      f"{row['val_loss']:<12.4f} {row['val_acc']:<12.2f} {row['time']:<10.2f}")
    
    return df_results, df_epoch_details


In [58]:
# Run all experiments for FashionMNIST
fashion_results = run_all_experiments()

# Save results to CSV
fashion_results.to_csv('fashionmnist_results.csv', index=False)

# Display results in a nice table format
print("\n\n" + "="*100)
print("FASHIONMNIST EXPERIMENT RESULTS")
print("="*100)
print(fashion_results.to_string(index=False))


Running FashionMNIST experiments with epochs 3 and 5 only...
Total experiments to run: 32

EXPERIMENT 1/32
Configuration:
  Model: resnet18
  Batch Size: 16
  Optimizer: SGD
  Learning Rate: 0.001
  Epochs: 2
  Epoch 1/2: Train Loss: 0.5152, Train Acc: 81.56%, Val Loss: 0.3903, Val Acc: 84.67%, Time: 35.64s
  Epoch 2/2: Train Loss: 0.3472, Train Acc: 87.24%, Val Loss: 0.3188, Val Acc: 88.00%, Time: 36.51s
  Test Accuracy: 88.19%
  Average training time per epoch: 36.07s

✓ Experiment 1 completed successfully!
  Final Test Accuracy: 88.19%
  Avg Time per Epoch: 36.07s

EXPERIMENT 2/32
Configuration:
  Model: resnet18
  Batch Size: 16
  Optimizer: SGD
  Learning Rate: 0.001
  Epochs: 3
  Epoch 1/3: Train Loss: 0.5139, Train Acc: 81.71%, Val Loss: 0.3564, Val Acc: 86.45%, Time: 37.22s
  Epoch 2/3: Train Loss: 0.3429, Train Acc: 87.34%, Val Loss: 0.3124, Val Acc: 88.48%, Time: 37.10s
  Epoch 3/3: Train Loss: 0.2872, Train Acc: 89.31%, Val Loss: 0.2865, Val Acc: 89.13%, Time: 34.95s
  Test 

AttributeError: 'tuple' object has no attribute 'to_csv'

In [59]:
fashion_results

(         Dataset     Model  Batch_Size Optimizer  Learning_Rate  Epochs  \
 0   FashionMNIST  resnet18          16       SGD         0.0010       2   
 1   FashionMNIST  resnet18          16       SGD         0.0010       3   
 2   FashionMNIST  resnet50          16       SGD         0.0010       2   
 3   FashionMNIST  resnet50          16       SGD         0.0010       3   
 4   FashionMNIST  resnet18          16       SGD         0.0001       2   
 5   FashionMNIST  resnet18          16       SGD         0.0001       3   
 6   FashionMNIST  resnet50          16       SGD         0.0001       2   
 7   FashionMNIST  resnet50          16       SGD         0.0001       3   
 8   FashionMNIST  resnet18          16      Adam         0.0010       2   
 9   FashionMNIST  resnet18          16      Adam         0.0010       3   
 10  FashionMNIST  resnet50          16      Adam         0.0010       2   
 11  FashionMNIST  resnet50          16      Adam         0.0010       3   
 12  Fashion

In [62]:
# For the tuple structure you showed
if isinstance(fashion_results, tuple) and len(fashion_results) >= 2:
    # Save the summary results (first DataFrame)
    fashion_results[0].to_csv('fashion_results_summary.csv', index=False)
    print("Saved summary to 'fashion_results_summary.csv'")
    
    # Save the detailed epoch results (second DataFrame)
    fashion_results[1].to_csv('fashion_results_detailed.csv', index=False)
    print("Saved detailed results to 'fashion_results_detailed.csv'")
    
    # Also create a combined file if you want
    summary_df = fashion_results[0]
    detailed_df = fashion_results[1]
    combined = pd.concat([summary_df, detailed_df], ignore_index=True)
    combined.to_csv('fashion_results_all.csv', index=False)
    print("Saved combined results to 'fashion_results_all.csv'")

Saved summary to 'fashion_results_summary.csv'
Saved detailed results to 'fashion_results_detailed.csv'
Saved combined results to 'fashion_results_all.csv'


In [64]:
import pandas as pd

# Read the CSV file
detailed_df = pd.read_csv('fashion_results_detailed.csv')

# Display basic info
print(f"File loaded successfully!")
print(f"Shape: {detailed_df.shape}")
print(f"Columns: {list(detailed_df.columns)}")

# Display the entire DataFrame
print("\n=== Complete Data ===")
print(detailed_df)

# Or display with formatting
print("\n=== Formatted Display ===")
print(detailed_df.to_string())


File loaded successfully!
Shape: (80, 12)
Columns: ['epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'time', 'exp_id', 'model', 'batch_size', 'optimizer', 'learning_rate', 'total_epochs']

=== Complete Data ===
    epoch  train_loss  train_acc  val_loss    val_acc       time  exp_id  \
0       1    0.515248  81.561905  0.390283  84.666667  35.637989       1   
1       2    0.347208  87.235714  0.318798  88.000000  36.507659       1   
2       1    0.513863  81.714286  0.356399  86.450000  37.219798       2   
3       2    0.342900  87.335714  0.312357  88.483333  37.098764       2   
4       3    0.287229  89.307143  0.286481  89.133333  34.954864       2   
..    ...         ...        ...       ...        ...        ...     ...   
75      1    0.931476  66.364286  0.622906  77.000000  51.743811      31   
76      2    0.544567  80.095238  0.464557  83.300000  51.492249      31   
77      1    0.914249  66.542857  0.565555  79.150000  51.833435      32   
78      2    0.5468